In [1]:
import pandas as pd
import pickle

In [2]:
train_df = pd.read_csv("train_lpp.csv")
train_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
train_df.shape

(614, 13)

In [4]:
train_df.dropna(inplace=True)

In [5]:
train_df.shape

(480, 13)

In [6]:
from sklearn.preprocessing import LabelEncoder 
cols = train_df.select_dtypes("object").columns.tolist()
cols

['Loan_ID',
 'Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Loan_Status']

In [7]:
label_encoders = {}
for i in cols:
    le = LabelEncoder()
    train_df[i] = le.fit_transform(train_df[i])
    label_encoders[i] = le

In [8]:
pickle.dump(label_encoders, open("rfclabelencoder.pkl", mode="wb"))

In [9]:
train_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,0,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,1,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,2,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,3,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1
5,4,1,1,2,0,1,5417,4196.0,267.0,360.0,1.0,2,1


In [10]:
X = train_df[["Gender","Married","ApplicantIncome","LoanAmount","Credit_History"]]

In [11]:
y = train_df[["Loan_Status"]]

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=42)

In [13]:
X_train

,Gender,Married,ApplicantIncome,LoanAmount,Credit_History
172,1,1,3522,81.0,1.0
288,0,0,4124,115.0,1.0
607,1,1,3987,157.0,1.0
55,1,1,2708,97.0,1.0
461,1,1,7740,128.0,1.0
...,...,...,...,...,...
141,1,0,5417,168.0,1.0
350,1,1,9083,228.0,1.0
445,1,1,3466,130.0,1.0
558,1,1,3887,162.0,1.0


In [14]:
y_train.value_counts()

Loan_Status
1              264
0              120
Name: count, dtype: int64

In [15]:
from sklearn.ensemble import RandomForestClassifier
dict_weights = {
    1:1,
    0:2
}
model = RandomForestClassifier(max_depth=4, random_state=5, class_weight=dict_weights, n_estimators=10)

In [16]:
model.fit(X_train, y_train)

/home/varun/Documents/Neoversity/14 MLOps/175 - Develop Web APIs using Flask/.venv/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(class_weight={0: 2, 1: 1}, max_depth=4, n_estimators=10,
                       random_state=5)

In [17]:
from sklearn.metrics import accuracy_score
pred_val = model.predict(X_val)
accuracy_score(y_val, pred_val)

0.8125

In [18]:
# Model Saving

import pickle
pickle_out = open("rfcclassifier.pkl", mode="wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [19]:
model_path = open("rfcclassifier.pkl", mode="rb")
model_upload = pickle.load(model_path)

In [20]:
accuracy_score(y_val, model_upload.predict(X_val))

0.8125